In [2]:
import numpy as np
from scipy.io import loadmat
from scipy.optimize import minimize
from sklearn import svm
import matplotlib.pyplot as plt

In [5]:
def preprocess():
    """ 
     Input:
     Although this function doesn't have any input, you are required to load
     the MNIST data set from file 'mnist_all.mat'.

     Output:
     train_data: matrix of training set. Each row of train_data contains 
       feature vector of a image
     train_label: vector of label corresponding to each image in the training
       set
     validation_data: matrix of training set. Each row of validation_data 
       contains feature vector of a image
     validation_label: vector of label corresponding to each image in the 
       training set
     test_data: matrix of training set. Each row of test_data contains 
       feature vector of a image
     test_label: vector of label corresponding to each image in the testing
       set
    """

    mat = loadmat('mnist_all.mat')  # loads the MAT object as a Dictionary

    n_feature = mat.get("train1").shape[1]
    n_sample = 0
    for i in range(10):
        n_sample = n_sample + mat.get("train" + str(i)).shape[0]
    n_validation = 1000
    n_train = n_sample - 10 * n_validation

    # Construct validation data
    validation_data = np.zeros((10 * n_validation, n_feature))
    for i in range(10):
        validation_data[i * n_validation:(i + 1) * n_validation, :] = mat.get("train" + str(i))[0:n_validation, :]

    # Construct validation label
    validation_label = np.ones((10 * n_validation, 1))
    for i in range(10):
        validation_label[i * n_validation:(i + 1) * n_validation, :] = i * np.ones((n_validation, 1))

    # Construct training data and label
    train_data = np.zeros((n_train, n_feature))
    train_label = np.zeros((n_train, 1))
    temp = 0
    for i in range(10):
        size_i = mat.get("train" + str(i)).shape[0]
        train_data[temp:temp + size_i - n_validation, :] = mat.get("train" + str(i))[n_validation:size_i, :]
        train_label[temp:temp + size_i - n_validation, :] = i * np.ones((size_i - n_validation, 1))
        temp = temp + size_i - n_validation

    # Construct test data and label
    n_test = 0
    for i in range(10):
        n_test = n_test + mat.get("test" + str(i)).shape[0]
    test_data = np.zeros((n_test, n_feature))
    test_label = np.zeros((n_test, 1))
    temp = 0
    for i in range(10):
        size_i = mat.get("test" + str(i)).shape[0]
        test_data[temp:temp + size_i, :] = mat.get("test" + str(i))
        test_label[temp:temp + size_i, :] = i * np.ones((size_i, 1))
        temp = temp + size_i

    # Delete features which don't provide any useful information for classifiers
    sigma = np.std(train_data, axis=0)
    index = np.array([])
    for i in range(n_feature):
        if (sigma[i] > 0.001):
            index = np.append(index, [i])
    train_data = train_data[:, index.astype(int)]
    validation_data = validation_data[:, index.astype(int)]
    test_data = test_data[:, index.astype(int)]

    # Scale data to 0 and 1
    train_data /= 255.0
    validation_data /= 255.0
    test_data /= 255.0

    return train_data, train_label, validation_data, validation_label, test_data, test_label

In [32]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [75]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [7]:
train_data, train_label, validation_data, validation_label, test_data, test_label = preprocess()

In [28]:
# number of classes
n_class = 10

# number of training samples
n_train = train_data.shape[0]

# number of features
n_feature = train_data.shape[1]

Y = np.zeros((n_train, n_class))
for i in range(n_class):
    Y[:, i] = (train_label == i).astype(int).ravel()

# Logistic Regression with Gradient Descent
W = np.zeros((n_feature + 1, n_class))

In [35]:
initialWeights = np.zeros((n_feature + 1, 1))
initialWeights.shape

(716, 1)

In [50]:
X = np.hstack((np.ones((train_data.shape[0], 1)), train_data))
X.shape

(50000, 716)

In [51]:
preds = sigmoid(np.dot(X, initialWeights))
preds.shape

(50000, 1)

In [52]:
labeli = Y[:, i].reshape(n_train, 1)
labeli.shape

(50000, 1)

In [53]:
error = np.mean((labeli * np.log(preds) + (1 - labeli) * np.log(1 - preds)))
error

-0.6931471805599453

In [54]:
error_grad = np.mean(np.dot(X.T, (preds - labeli)))
error_grad

2934.8328212290603

In [76]:
y = np.dot(X, W)
y.shape

(50000, 1)

## Multi Class

In [144]:
X = np.hstack((np.ones((train_data.shape[0], 1)), train_data))
X.shape

(50000, 716)

In [119]:
Y = np.zeros((n_train, n_class))
for i in range(n_class):
    Y[:, i] = (train_label == i).astype(int).ravel()
Y.shape

(50000, 10)

In [145]:
W = np.random.rand(n_feature + 1, n_class)
W.shape

(716, 10)

In [146]:
theta = softmax(np.dot(X, W))
theta.shape

(50000, 10)

In [142]:
np.argmax(theta, axis=1).

(50000,)

In [128]:
(Y*np.log(theta))

array([[-106.63011492,   -0.        ,   -0.        , ...,   -0.        ,
          -0.        ,   -0.        ],
       [ -47.08100045,   -0.        ,   -0.        , ...,   -0.        ,
          -0.        ,   -0.        ],
       [ -93.88286663,   -0.        ,   -0.        , ...,   -0.        ,
          -0.        ,   -0.        ],
       ...,
       [  -0.        ,   -0.        ,   -0.        , ...,   -0.        ,
          -0.        , -116.42860329],
       [  -0.        ,   -0.        ,   -0.        , ...,   -0.        ,
          -0.        , -120.59530082],
       [  -0.        ,   -0.        ,   -0.        , ...,   -0.        ,
          -0.        , -123.92504836]])

In [136]:
np.sum(Y*np.log(theta))

-5309845.472609932

In [137]:
np.dot(X.T, theta - labeli)

array([[-4.94800000e+03, -4.94800000e+03, -4.94800000e+03, ...,
        -4.94800000e+03, -4.94800000e+03, -4.94800000e+03],
       [ 1.84777721e-53,  7.57476541e-50,  2.47793392e-50, ...,
         5.62142724e-48,  7.03651679e-53,  7.60788440e-51],
       [ 3.54447486e-52,  1.10703791e-48,  4.11344150e-49, ...,
         4.87540989e-47,  1.47342565e-51,  1.53789799e-49],
       ...,
       [-1.90196078e+00, -1.90196078e+00, -1.90196078e+00, ...,
        -1.90196078e+00, -1.90196078e+00, -1.90196078e+00],
       [-1.96078431e-01, -1.96078431e-01, -1.96078431e-01, ...,
        -1.96078431e-01, -1.96078431e-01, -1.96078431e-01],
       [ 5.88049521e-54,  3.55840044e-52,  7.26981821e-51, ...,
         3.81155923e-51,  4.26142419e-55,  3.20590478e-52]])

In [138]:
np.dot(X.T, theta - labeli).ravel()

array([-4.94800000e+03, -4.94800000e+03, -4.94800000e+03, ...,
        3.81155923e-51,  4.26142419e-55,  3.20590478e-52])

In [143]:
clf = svm.SVC(kernel='linear')
clf.fit(train_data, train_label.flatten())
print(f'Training set Accuracy: {100*clf.score(train_data, train_label)}%')
print(f'Validation set Accuracy: {100*clf.score(validation_data, validation_label)}%')
print(f'Testing set Accuracy: {100*clf.score(test_data, test_label)}%')

Training set Accuracy: 97.286%
Validation set Accuracy: 93.64%
Testing set Accuracy: 93.78%
